<a href="https://colab.research.google.com/github/yukinaga/gnn/blob/main/section_4/02_gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCNの実装
Graph Convolutional Networks（GCN）を実装します。 
多数のグラフを持つデータセットを訓練データに使い、ミニバッチ法により学習を行います。   
学習にGPUを利用するので、「編集」→「ノートブックの設定」の「ハードウェアアクセラレーター」で「GPU」を選択しましょう。

## Google ドライブとの連携  
今回はライブラリのサイズが大きく毎回インストールするのが大変なので、Googleドライブに保存します。  
まずは以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Googleドライブ上のパスを指定します。

In [ ]:
dir_name = "Live/gnn_live"  # 好きなパスを設定してください
package_path = "/content/drive/MyDrive/" + dir_name + "/packages/"

## PyTorch Geometricのインストール
GNN用のライブラリ「PyTorch Geometric」、および関連ライブラリをGoogle Driveのパスを指定してインストールします。  
既にGoogle Driveにこれらのライブラリがインストール済みであれば、以下のセルのコードを実行する必要はありません。

In [ ]:
!pip install --no-cache-dir torch-geometric torch-sparse torch-scatter -t $package_path

Google Driveに保存したパッケージをシステムに追加します。  

In [ ]:
import sys

sys.path.append(package_path)  

## データセットの読み込み
TUDatasetから、188のグラフが含まれるデータセット「MUTAG」を読み込みます。  


In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root="/tmp/MUTAG", name="MUTAG")

dataset = dataset.shuffle()  # データセットをシャッフル
dataset_train = dataset[:140]  # 訓練用データセット
dataset_test = dataset[140:]  # テスト用データセット

batch_size = 64  # バッチサイズ
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

## モデルの構築
GCNのモデルを構築します。  
層の実装には、`GCNConv()`を利用しますが、以下のように設定します。  
```
GCNConv(入力の特徴量数, 出力の特徴量数)
```  
https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv  
今回は、全結合層の前にニューロンをランダムに削除する「ドロップアウト」を導入します。  
ドロップアウトを導入することで、未知のデータに対してより頑強になるようにモデルを訓練することができます。    
https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html  

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

n_h = 64  # 中間層における特徴量の数

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, n_h)
        self.conv2 = GCNConv(n_h, n_h)
        self.conv3 = GCNConv(n_h, n_h)
        self.fc = nn.Linear(n_h, dataset.num_classes)  # 全結合層

        self.relu = nn.ReLU()  # ReLU
        self.dropout = nn.Dropout(p=0.5)  # ドロップアウト:(p=ドロップアウト率)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        batch = data.batch

        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        x = self.relu(x)
        x = self.conv3(x, edge_index)

        # 全てのノードで各特徴量の平均をとる
        x = global_mean_pool(x, batch)  # (バッチサイズ, 特徴量の数)に変換

        x = self.dropout(x)
        x = self.fc(x)
        
        return x

net = GCN()
net.cuda()  #GPU対応

## 学習

モデル評価用の関数を用意します。

In [ ]:
def eval(loader):
    correct = 0  # 正解数

    for data in loader:
        data = data.cuda()  # GPU対応
        out = net(data)  
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())

    return correct/len(loader.dataset)  # 正解率

訓練データを使い、モデルを訓練します。

In [ ]:
from torch import optim

# 交差エントロピー誤差関数
loss_fnc = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(net.parameters())

for epoch in range(200):
    # 訓練
    net.train()  # 訓練モード
    for data in loader_train: 
        data = data.cuda()  # GPU対応

        optimizer.zero_grad()  # ①勾配の初期化
        out = net(data)  # ②順伝播により予測値を得る
        loss = loss_fnc(out, data.y)  # ③予測値と正解値から誤差を計算

        loss.backward()  # ④誤差からバックプロパゲーションにより勾配を計算
        optimizer.step()  # ⑤最適化アルゴリズムによりパラメータを更新

    # 評価
    net.eval()  # 評価モード
    acc_train = eval(loader_train)
    acc_test = eval(loader_test)
    print("Epoch:", epoch,
          "acc_train:", str(acc_train*100) + "%",
          "acc_test:", str(acc_test*100) + "%")

## モデルの評価
訓練済みのモデルを評価します。

In [ ]:
net.eval()  # 評価モード
acc_test = eval(loader_test)
print("accuracy:", str(acc_test*100) + "%")